In [32]:
# 1) استيراد المكتبات
import pandas as pd
import numpy as np

In [33]:
# 2) قراءة ملف العقارات
df = pd.read_csv("C:/Users/tf/Desktop/مشروع 1/Real Estate Data.csv")

In [34]:
# 3) تنظيف وتحويل التاريخ
df['instance_date'] = pd.to_datetime(
    df['instance_date'],
    errors='coerce',
    dayfirst=True
)

# حذف الصفوف ذات التاريخ غير الصالح
df = df.dropna(subset=['instance_date'])

# تصفية السنوات المطلوبة
df = df[
    (df['instance_date'].dt.year >= 2015) &
    (df['instance_date'].dt.year <= 2025)
]

In [35]:
# 4) اختيار الأعمدة الجوهرية فقط
cols = [
    'instance_date',
    'area_name_en',
    'property_type_en',
    'actual_worth',
    'meter_sale_price',
    'procedure_area'
]

df = df[cols].copy()

In [36]:
# 5) تحويل الأعمدة الرقمية
num_cols = ['actual_worth', 'meter_sale_price', 'procedure_area']

for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# حذف الصفوف التي تفتقد قيمًا أساسية
df = df.dropna(subset=num_cols)

In [37]:
# 6) تنظيف أسماء المناطق 
df['area_name_en'] = (
    df['area_name_en']
    .str.strip()
    .str.lower()
)

In [38]:
# 7) تشخيص جودة البيانات 
missing_report = pd.DataFrame({
    'missing_count': df.isna().sum(),
    'missing_percent': (df.isna().sum() / len(df)) * 100
}).sort_values(by='missing_percent', ascending=False)

print("=== Missing Values Report ===")
print(missing_report)

=== Missing Values Report ===
                  missing_count  missing_percent
instance_date                 0              0.0
area_name_en                  0              0.0
property_type_en              0              0.0
actual_worth                  0              0.0
meter_sale_price              0              0.0
procedure_area                0              0.0


In [39]:
# 8) فحص القيم الشاذة 
print("=== Descriptive Statistics ===")
print(df[num_cols].describe())

=== Descriptive Statistics ===
       actual_worth  meter_sale_price  procedure_area
count  5.711790e+05      5.711790e+05    5.711790e+05
mean   2.624554e+06      1.242635e+04    8.975005e+02
std    6.046982e+06      1.443333e+04    1.221537e+05
min    1.000000e+00      0.000000e+00    1.300000e-01
25%    7.230400e+05      6.807590e+03    7.014000e+01
50%    1.323000e+06      1.000255e+04    1.200000e+02
75%    2.359000e+06      1.512067e+04    2.322600e+02
max    9.997125e+07      2.034192e+06    6.414803e+07


In [40]:
# 9) 
df = df[
    (df['meter_sale_price'] > 0) &
    (df['procedure_area'] > 0)
]

In [41]:
# 10) إنشاء المفاتيح الزمنية
df['year'] = df['instance_date'].dt.year
df['month'] = df['instance_date'].dt.month

# مفتاح شهري للدمج
df['year_month'] = df['instance_date'].dt.to_period('M').astype(str)

In [42]:

# 11) فحص قوة مفاتيح الدمج 
print("عدد الأشهر:", df['year_month'].nunique())
print("عدد المناطق:", df['area_name_en'].nunique())

print("أكثر المفاتيح تكرارًا:")
print(df[['year_month', 'area_name_en']].value_counts().head(10))


عدد الأشهر: 99
عدد المناطق: 195
أكثر المفاتيح تكرارًا:
year_month  area_name_en          
2022-09     business bay              1327
2022-08     business bay              1292
2022-10     business bay              1237
2023-02     wadi al safa 5            1189
2022-11     business bay              1176
2023-01     business bay              1136
2023-02     al barsha south fourth    1120
2023-03     al barsha south fourth    1114
2022-12     al merkadh                1015
2022-11     marsa dubai               1008
Name: count, dtype: int64


In [43]:
# 12) قاموس توحيد تهجئات المناطق
area_fix = {
    'dubai-marina': 'dubai marina',
    'dubaai marina': 'dubai marina'
}

df['area_name_en'] = df['area_name_en'].replace(area_fix)

In [44]:
# 13) التجميع الشهري حسب المنطقة ونوع العقار
real_estate_month = (
    df
    .groupby(['year_month', 'area_name_en', 'property_type_en'])
    .agg(
        avg_meter_price=('meter_sale_price', 'mean'),
        avg_actual_worth=('actual_worth', 'mean'),
        avg_area=('procedure_area', 'mean'),
        transactions_count=('meter_sale_price', 'count')
    )
    .reset_index()
)

In [45]:
# 14) حفظ الناتج النهائي
real_estate_month.to_csv(
    "C:/Users/tf/Desktop/مشروع 1/Data after cleaning/real_estate_month.csv",
    index=False
)

In [46]:
# 15) فحص نهائي
print(real_estate_month.head())
print(real_estate_month.info())


  year_month     area_name_en property_type_en  avg_meter_price  \
0    2015-01         abu hail         Building          8987.42   
1    2015-01         abu hail             Land          6772.56   
2    2015-01         abu hail            Villa          6404.26   
3    2015-01   al aweer first         Building           538.20   
4    2015-01  al aweer second         Building           269.10   

   avg_actual_worth  avg_area  transactions_count  
0         1500000.0    166.90                   1  
1         3850000.0    568.47                   1  
2         1800000.0    428.12                   4  
3         6000000.0  11148.36                   1  
4          500000.0   1858.04                   1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22716 entries, 0 to 22715
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   year_month          22716 non-null  object 
 1   area_name_en        22716